In [4]:
import time
import hmac
import hashlib
import base64
import requests
import numpy as np

In [36]:
class KCClient():
    def __init__(self, agent):
        self.API_SECRET = '26089ed9-e0a9-4c7e-b355-b9cd41e2bd6d'
        self.url = 'https://api.kucoin.com'
        self.agent = agent
        self.session = requests.session()
    
    def get_ticker_price(self, symbol):
        request = Request('{}/open/tick?symbol={}'.format(self.url, symbol))
        return urlopen(request).read()
    
    def order_asset(self, data):
        """
        Place an order on an asset. Expect data with the following parameters:
        symbol: pairing indicating ticker asset (ex: ETH-BTC)
        type: BUY or SELL
        amount: amount of assets in transaction
        price: how much to buy/sell asset for
        
        return: response from api
        """
        endpoint = '/v1/order'
        params = self.generate_params(data)
        headers = self.generate_header(endpoint, data)

        self.session.headers.update(headers)
        uri = '{}{}?{}'.format(self.url, endpoint, params)
        
        response = getattr(self.session, 'post')(uri)
        return response.json()
    
#     def cancel_order(self, data):
    
    def generate_header(self, endpoint, data):
        nonce = int(time.time() * 1000)
        return {
            'Accept': 'application/json',
            'User-Agent': 'python-kucoin',
            'HTTP_ACCEPT_LANGUAGE': 'en-US',
            'Accept-Language': 'en-US',
            'KC-API-KEY': '5a5403f96d581b54c6e60c09',
            'KC-API-NONCE': str(nonce),
            'KC-API-SIGNATURE': self.generate_signature(endpoint, data, nonce),
        }
    
    def generate_signature(self, endpoint, data, nonce):
        """Generate the call signature
        :param data:
        :param nonce:
        :return: signature string
        """
        query_string = self.generate_params(data)
        sig_str = ("{}/{}/{}".format(endpoint, nonce, query_string)).encode('utf-8')
        m = hmac.new(self.API_SECRET.encode('utf-8'), base64.b64encode(sig_str), hashlib.sha256)
        return m.hexdigest()
    
    def generate_params(self, data):
        strs = []
        for key in sorted(data):
            strs.append('{}={}'.format(key, data[key]))
        return '&'.join(strs)

In [3]:
trader = KCTrader('')
trader.order_asset({
    'symbol': 'ETH-BTC', 
    'type': 'BUY', 
    'amount': '1', 
    'price': '0.000001'})

NameError: name 'KCTrader' is not defined

In [6]:
def generate_random_portfolio(num_assets):
    a = np.random.rand(num_assets)
    a /= a.sum()
    return a

In [7]:
generate_random_portfolio(10)

array([ 0.17324032,  0.18434495,  0.17218054,  0.06700218,  0.12021989,
        0.00072943,  0.03328452,  0.14985426,  0.08642177,  0.01272215])

In [20]:
w0 = np.array([1,0,0,0,0,0,0,0,0,0])
w1 = np.array([0.5, 0.25, 0.25, 0, 0,0,0,0,0,0])
p0 = 1.
v0 = generate_random_portfolio(10)
v1 = generate_random_portfolio(10)

In [21]:
s0 = np.divide(w0*p0, v0)
p1 = np.sum(s0.dot(v1))

In [22]:
s1 = np.divide(w1*p1, v1)

In [29]:
d_s1 = s1 - s0

In [30]:
d_s1

array([-97.55817561,  34.19412241,   3.03711512,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ])

In [31]:
s0

array([ 195.11635121,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
          0.        ,    0.        ])

In [32]:
s1

array([ 97.55817561,  34.19412241,   3.03711512,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ])

In [47]:
class KCTrader():
    def __init__(self, num_assets, p_init, commission_rate, min_trade_amount=1e-4):
        self.w_prev = np.eye(num_assets)[0]
        self.p_prev = p_init
        self.commission_rate = commission_rate
        self.client = KCTrader()
        self.min_trade_amount = min_trade_amount
        
    @static
    def calculate_mu(w1, w0, commission_rate):
    """
    @:param w1: target portfolio vector, first element is btc
    @:param w0: rebalanced last period portfolio vector, first element is btc
    @:param commission_rate: rate of commission fee, proportional to the transaction cost
    """
    mu0 = 1
    mu1 = 1 - 2*commission_rate + commission_rate ** 2
    while abs(mu1-mu0) > 1e-10:
        mu0 = mu1
        mu1 = (1 - commission_rate * w0[0] -
            (2 * commission_rate - commission_rate ** 2) *
            np.sum(np.maximum(w0[1:] - mu1*w1[1:], 0))) / \
            (1 - commission_rate * w1[0])
    return mu1
 
    def process_w(w):
        """
        Since w vector values can be very small i.e 0.1e-10 and there is a minimum trading limit
        we need to process these values to be 0 and make sure that sum(w) still keeps at 1.
        Args:
        - w: raw w vector values
        Return:
        tapered and normalized w vector.
        """
        for i in xrange(len(w)):
            if w[i] < self.min_trade_amount:
                w[i] = 0.
        w /= np.sum(w)
        return w

    def execute_action(self, w, v_prev, v):
        w = process_w(w)
        s_prev = np.divide(self.w_prev*self.p_prev, v_prev)
        p_prime = np.sum(s_prev.dot(v))
        mu = calculate_mu(w, self.w_prev, self.commission_rate)
        p = mu*p_prime
        # current values 
        s = np.divide(w*p, v)
        # calculate next trading asset values
        d_s = s - s_prev
        
        for i, asset in enumerate(self.assets):
            if d_s[i] == 0: continue
            self.client.order_asset(
                {'symbol': asset, 
                 'type': 'BUY' if d_s[i] > 0 else 'SELL', 
                 'amount': d_s[i], 
                 'price': client.get_price(asset)})

        
        

        

IndentationError: expected an indented block (<ipython-input-47-db6e6b2acd04>, line 15)

In [40]:
w = np.array([0.00000000e+00,1.51549227e-08,9.99698400e-01,1.46747169e-07,
              1.60798996e-11,7.99844383e-06,2.55209079e-06,1.02792634e-04,
              1.08958578e-12,1.30443550e-06,1.86839126e-04,2.19902391e-14
             ])

In [52]:
np.sum(process_w(w))

1.0